In [11]:
import pandas as pd
import pickle
from pathlib import Path

In [20]:
year = 2022
month = 2
taxi_type = 'yellow'

input_data = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year}-{month:02}.parquet'
output_dir = Path('output')
output_dir.mkdir(parents=True, exist_ok=True)
output_data = output_dir / f'{taxi_type}_tripdata_{year}-{month:02}.parquet'


with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)


In [ ]:
# categorical = ['PULocationID', 'DOLocationID']

# def read_data(filename):
#     df = pd.read_parquet(filename)
    
#     df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
#     df['duration'] = df.duration.dt.total_seconds() / 60

#     df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

#     df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

#     #Create an artificial ride_id
#     df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
#     return df


# def apply_model(input_file, model, output_file):
#     df = read_data(input_file)
    
#     X = dv.transform(df[categorical].to_dict(orient='records'))
#     y_pred = model.predict(X)

#     df_result = pd.DataFrame()
#     df_result['ride_id'] = df.ride_id
#     df_result['tpep_pickup_datetime'] = df.tpep_pickup_datetime
#     df_result['tpep_dropoff_datetime'] = df.tpep_dropoff_datetime
#     df_result['PULocationID'] = df.PULocationID
#     df_result['DOLocationID'] = df.DOLocationID
#     df_result['actual_duration'] = df.duration
#     df_result['prediction_duration'] = y_pred
#     df_result['different'] = df_result.prediction_duration - df_result.actual_duration

#     # Save the result
#     df_result.to_parquet(
#         output_file,
#         engine='pyarrow',
#         compression=None,
#         index=False
#     )   
        


In [27]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    #Create an artificial ride_id
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    return df


def apply_model(input_file, model, output_file):
    df = read_data(input_file)
    
    X = dv.transform(df[categorical].to_dict(orient='records'))
    y_pred = model.predict(X)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df.ride_id
    df_result['prediction_duration'] = y_pred


    # Save the result
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

    return df_result   
        


In [28]:
result = apply_model(input_data, model, output_data) # 1.5 min


In [29]:
result.head()

,ride_id,prediction_duration
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904


In [30]:
!ls -lh output/

total 188M
-rw-r--r-- 1 Trung 197121  58M Jun 17 20:24 yellow_tripdata_2022-02.parquet
-rw-r--r-- 1 Trung 197121 131M Jun 17 17:33 yellow_tripdata_2023-02.parquet


In [31]:
result_march = pd.read_parquet('output/yellow_tripdata_2022-03.parquet')
result_march.prediction_duration.mean()

12.758556818790902

### Command to convert ipynb to py
- **pip install nbconvert**
- **jupyter nbconvert --to script score.ipynb**
